In [147]:
import re
import sys
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
from urllib.parse import quote
from sklearn import preprocessing
from urllib.request import urlopen
from matplotlib import pyplot as plt


pd.set_option('display.max_columns', 49)
%matplotlib inline

In [148]:
path = r"/Users/stanislav/Мои документы/ml/1. developer/"
path + "flats_25.10.2018.csv"

'/Users/stanislav/Мои документы/ml/1. developer/flats_25.10.2018.csv'

In [149]:
data = pd.read_csv(path + "flats_25.10.2018.csv", sep=";",low_memory=False, encoding="utf-8")
data.head()

,Cart_Number,Feature,Rooms_Number,Object_Type,Object_Type_ID,Settlement,Settlement_ID,District,District_Id,Street,Street_Id,House_Number,Metro,Metro_ID,Metro_m,Flats_Plan,Flats_Plan_ID,Stor,Storeys,Wall,Wall_ID,Space_Total,Space_Living,Space_Kitchen,...,Lodgee_Num,Sost,Sost_ID,Clozet,Clozet_ID,lat,Long,Date_Postr,Date_Create,Childrens_leisure,Childrens_Distance,Childrens_Count,Schools,Schools_Distance,Schools_Count,Kindergarten,Kindergarten_Distance,Kindergarten_Count,Products,Products_Distance,Products_Count,Sport,Sport_Distance,Sport_Count
0,100316920,Продана не нами,1,Улучшенной планировки,3,Новосибирск,1,Заельцовский,3,Красный проспект,901,181,Заельцовская,283,2890,NaN,0,13,17,Кирпич,2,"54,1",18,"21,2",...,1,Самоотделка,9,Совмещенный,8,"55,0725","82,9069",2010,2009-04-21 00:00:00.000,"Спектр, центр развития и творчества молодежи",337.0,12.0,"Сибирский Кадетский Корпус, школа-интернат",142.0,3.0,"Берёзка, детский сад №272 комбинированного вида",278.0,5.0,"У соседа, продовольственный магазин",113.0,17.0,"Солнечный, детский бассейн",364.0,0.0
1,100316924,Продана не нами,1,Улучшенной планировки,3,Новосибирск,1,Заельцовский,3,Красный проспект,901,181,Заельцовская,283,2890,NaN,0,10,17,Кирпич,2,"54,5",18,"21,1",...,1,Требует ремонта,4,Раздельный,9,"55,0725","82,9069",2010,2009-08-17 00:00:00.000,"Спектр, центр развития и творчества молодежи",337.0,12.0,"Сибирский Кадетский Корпус, школа-интернат",142.0,3.0,"Берёзка, детский сад №272 комбинированного вида",278.0,5.0,"У соседа, продовольственный магазин",113.0,17.0,"Солнечный, детский бассейн",364.0,0.0
2,10103978,Продана не нами,1,Улучшенной планировки,3,Новосибирск,1,Центральный,10,Красный проспект,900,181,Заельцовская,283,2830,NaN,0,8,17,Кирпич,2,37,0,0,...,0,NaN,0,NaN,0,"55,0725","82,9068",2008,2016-09-27 10:07:31.490,"Спектр, центр развития и творчества молодежи",337.0,12.0,"Сибирский Кадетский Корпус, школа-интернат",142.0,3.0,"Берёзка, детский сад №272 комбинированного вида",278.0,5.0,"У соседа, продовольственный магазин",113.0,17.0,"Солнечный, детский бассейн",364.0,0.0
3,10100878,Продана не нами,1,Улучшенной планировки,3,Новосибирск,1,Центральный,10,Красный проспект,900,181,Заельцовская,283,2830,NaN,0,2,17,Кирпич,2,42,0,0,...,0,NaN,0,NaN,0,"55,0725","82,9068",2008,2018-07-28 08:22:15.223,"Спектр, центр развития и творчества молодежи",337.0,12.0,"Сибирский Кадетский Корпус, школа-интернат",142.0,3.0,"Берёзка, детский сад №272 комбинированного вида",278.0,5.0,"У соседа, продовольственный магазин",113.0,17.0,"Солнечный, детский бассейн",364.0,0.0
4,10104701,Продана не нами,1,Улучшенной планировки,3,Новосибирск,1,Центральный,10,Красный проспект,900,181,Заельцовская,283,2830,NaN,0,13,17,Кирпич,2,28,0,0,...,0,Отличное,6,NaN,0,"55,0725","82,9068",2008,2017-06-23 10:36:04.610,"Спектр, центр развития и творчества молодежи",337.0,12.0,"Сибирский Кадетский Корпус, школа-интернат",142.0,3.0,"Берёзка, детский сад №272 комбинированного вида",278.0,5.0,"У соседа, продовольственный магазин",113.0,17.0,"Солнечный, детский бассейн",364.0,0.0


### Формируем полные адреса квартир

In [150]:
data["House_Number"]=data["House_Number"].str.replace(" ","").str.lower()

In [151]:
data["full_address_home"] = "Новосибирск, " + data["District"] + ", " + data["Street"] + ", " + data["House_Number"]
data["full_address_home"] = data["full_address_home"].str.replace(" ","+")
df_full_address["full_address_home"] = data["full_address_home"].drop_duplicates()
df_full_address.head()

0      Новосибирск,+Заельцовский,+Красный+проспект,+181
2       Новосибирск,+Центральный,+Красный+проспект,+181
38    Новосибирск,+Заельцовский,+Красный+проспект,+2...
39     Новосибирск,+Заельцовский,+Красный+проспект,+232
82          Новосибирск,+Заельцовский,+Шевцовой+2-я,+37
Name: full_address_home, dtype: object

In [160]:
def add_info_geo(data,address):
    latit = list()
    longit = list()
    for i in range(len(data)):
        if (i%1000==0):
            print(i)
            print("https://geocode-maps.yandex.ru/1.x/?geocode="+quote(data[address][i]))
        main_url = "https://geocode-maps.yandex.ru/1.x/?geocode="+quote(data[address][i])
        html = urlopen(main_url)
        soup = BeautifulSoup(html.read(), "html.parser")
        featuremember = soup.find("featuremember")     
        geo_cord = featuremember.find("pos").text.split(" ")  
        latit.append(geo_cord[1])
        longit.append(geo_cord[0])
        
    data["latit"] = latit
    data["Longit"] = longit
    return data

In [161]:
full_address = add_info_geo(df_full_address,"full_address_home")
full_address.head(10)

0
https://geocode-maps.yandex.ru/1.x/?geocode=%D0%9D%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA%2C%2B%D0%97%D0%B0%D0%B5%D0%BB%D1%8C%D1%86%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9%2C%2B%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D1%8B%D0%B9%2B%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82%2C%2B181


KeyError: 1

### GPS координаты станций Новосибирского метро

In [6]:
data["Metro"].unique()

array(['Заельцовская', nan, 'Правый берег', 'Площадь Маркса',
       'Студенческая', 'Берёзовая роща', 'Октябрьская', 'Золотая Нива',
       'Речной вокзал', 'Площадь Гарина-Михайловского', 'Площадь Ленина',
       'Маршала Покрышкина', 'Красный проспект', 'Сибирская',
       'Гагаринская', 'Левый берег'], dtype=object)

In [146]:
metro_geo = {"Заельцовская" : ["55.059410", "82.912488"],
"Гагаринская" : ["55.051071", "82.914770"],
"Красный проспект" : ["55.041168", "82.917429"],
"Площадь Ленина" : ["55.030003", "82.920492"],
"Октябрьская" : ["55.019134", "82.939303"],
"Речной вокзал" : ["55.008573", "82.937821"],
"Студенческая" : ["54.989140", "82.906649"],
"Площадь Маркса" : ["54.983050", "82.893013"],
"Площадь Гарина-Михайловского" : ["55.035385", "82.898780"],
"Сибирская" : ["55.042323", "82.919298"],
"Маршала Покрышкина" : ["55.043768", "82.935539"],
"Берёзовая роща" : ["55.043355", "82.953056"],
"Золотая Нива" : ["55.037969", "82.976215"]}
metro_geo = pd.DataFrame(metro_geo)
metro_geo.head()

,Заельцовская,Гагаринская,Красный проспект,Площадь Ленина,Октябрьская,Речной вокзал,Студенческая,Площадь Маркса,Площадь Гарина-Михайловского,Сибирская,Маршала Покрышкина,Берёзовая роща,Золотая Нива
0,55.059410,55.051071,55.041168,55.030003,55.019134,55.008573,54.989140,54.983050,55.035385,55.042323,55.043768,55.043355,55.037969
1,82.912488,82.914770,82.917429,82.920492,82.939303,82.937821,82.906649,82.893013,82.898780,82.919298,82.935539,82.953056,82.976215


### Парсинг адреса учерждений с сайта
https://neagent.info/novosibirsk/infrastructure/ <br />
Инфраструктура Новосибирска, Метро, станции, Университеты, Детские сады, Школы, Торговые центры, Магазины, торговые сети

* **Образовательные учреждения**

In [81]:
schools = pd.DataFrame(columns=["name","address"])
main_school_url = "https://neagent.info/novosibirsk/school/"
site_list = ["licei","chastnye","sosch","gimnazii"]    

In [109]:
name = list()
adr = list()
for l in site_list:
    url = main_school_url + l
    html = urlopen(url)
    soup = BeautifulSoup(html.read(), "html.parser")
    table = soup.find("table", {"class": "district-markerlist"})
    obj = table.findAll("div", {"class" : "params-marker-desc"})
    for i in obj:
        address = i.find_all("p")
        name.append(address[1].text.lstrip())
        adr.append(address[0].text.lstrip())

school = pd.DataFrame({"name":name, "address":adr})
school["address_for_url"] = "Новосибирск,+"+school["address"].str.replace(" ","+")
school.head()

,name,address,address_for_url
0,МАОУ Лицей №6,"г. Бердск, ул. Горького, 7/3","Новосибирск,+г.+Бердск,+ул.+Горького,+7/3"
1,МАОУ Лицей №7,"г. Бердск, ул. Ленина, 104","Новосибирск,+г.+Бердск,+++ул.+Ленина,+104"
2,МБОУ Аэрокосмический лицей имени Ю.В. Кондратюка,"Дзержинский, ул. Индустриальная, 4а","Новосибирск,+Дзержинский,+ул.++Индустриальная,+4а"
3,МБОУ Лицей №113,"Дзержинский, ул. Бориса Богаткова, 241/1","Новосибирск,+Дзержинский,+ул.++Бориса+Богатков..."
4,МБОУ Лицей №9,"Железнодорожный, ул. 1905 года, 41","Новосибирск,+Железнодорожный,+ул.+1905+года,+41"


In [110]:
# Координаты
school = add_info_geo(school,"address_for_url")
school.head()

0


,name,address,address_for_url,latit,Longit
0,МАОУ Лицей №6,"г. Бердск, ул. Горького, 7/3","Новосибирск,+г.+Бердск,+ул.+Горького,+7/3",54.759867,83.112022
1,МАОУ Лицей №7,"г. Бердск, ул. Ленина, 104","Новосибирск,+г.+Бердск,+++ул.+Ленина,+104",54.756911,83.113433
2,МБОУ Аэрокосмический лицей имени Ю.В. Кондратюка,"Дзержинский, ул. Индустриальная, 4а","Новосибирск,+Дзержинский,+ул.++Индустриальная,+4а",55.05547,82.970744
3,МБОУ Лицей №113,"Дзержинский, ул. Бориса Богаткова, 241/1","Новосибирск,+Дзержинский,+ул.++Бориса+Богатков...",55.039976,82.977931
4,МБОУ Лицей №9,"Железнодорожный, ул. 1905 года, 41","Новосибирск,+Железнодорожный,+ул.+1905+года,+41",55.042504,82.90196


In [111]:
school.to_csv(path + "nov_schools.csv")

* **Университеты**

In [120]:
main_univer_url = "https://neagent.info/novosibirsk/university/"
short = list()
full = list()
html = urlopen(main_univer_url)
soup = BeautifulSoup(html.read(), "html.parser")
div = soup.find("div", {"class" : "contentblock"})
table = div.find("table")
tr = table.findAll("tr")

for i in tr:
    short_name = i.find("h2")
    full_name = i.find("p")
    if short_name and full_name:
        short.append(short_name.text.lstrip())
        full.append(full_name.text.lstrip())

univer = pd.DataFrame({"short_name":short, "full_name":full})
univer["address_for_url"] = univer["short_name"].str.replace(" ","+")
univer.head()

,short_name,full_name,address_for_url
0,СГУВТ,Сибирский государственный университет водного ...,СГУВТ
1,СГУГиТ,Сибирский государственный университет геосисте...,СГУГиТ
2,НВИ,Новосибирский военный институт им. генерала ар...,НВИ
3,НГУАДИ,Новосибирский государственный университет архи...,НГУАДИ
4,НГТУ,Новосибирский государственный технический унив...,НГТУ


In [125]:
univer["naddress"] = ""
univer["naddress"][0] = "ул. Щетинкина, 33, Новосибирск"
univer["naddress"][1] = "ул. Плахотного, 10"
univer["naddress"][2] = "ул. Ключ-Камышенское Плато, 6/2"
univer["naddress"][3] = "Красный просп., 38"
univer["naddress"][4] = "просп. Карла Маркса, 20"
univer["naddress"][5] = "Вилюйская ул., 28"
univer["naddress"][6] = "Советская ул., 31"
univer["naddress"][7] = "просп. Карла Маркса, 26"
univer["naddress"][8] = "Ленинградская ул., 113"
univer["naddress"][9] = "Нижегородская ул., 6"
univer["naddress"][10] = "ул. Дуси Ковальчук, 191"
univer["naddress"][11] = "ул. Медкадры, 8"
univer["naddress"][12] = "Каменская ул., 52/1,"
univer["naddress"][13] = "ул. Пирогова, 1, Советский район, микрорайон Академгородок"
univer["naddress"][14] = "ул. Революции, 6"
univer["naddress"][15] = "Красный просп., 52"
univer["naddress"][16] = "ул. Добролюбова, 160"
univer["naddress"][17] = "ул. Кирова, 86"

In [126]:
# Координаты
univer["address_for_url"] = univer["naddress"].str.replace(" ","+")
univer.head()

,short_name,full_name,address_for_url,naddress
0,СГУВТ,Сибирский государственный университет водного ...,"ул.+Щетинкина,+33,+Новосибирск","ул. Щетинкина, 33, Новосибирск"
1,СГУГиТ,Сибирский государственный университет геосисте...,"ул.+Плахотного,+10","ул. Плахотного, 10"
2,НВИ,Новосибирский военный институт им. генерала ар...,"ул.+Ключ-Камышенское+Плато,+6/2","ул. Ключ-Камышенское Плато, 6/2"
3,НГУАДИ,Новосибирский государственный университет архи...,"Красный+просп.,+38","Красный просп., 38"
4,НГТУ,Новосибирский государственный технический унив...,"просп.+Карла+Маркса,+20","просп. Карла Маркса, 20"


In [164]:
univer = add_info_geo(univer,"address_for_url")
univer.head()

0
https://geocode-maps.yandex.ru/1.x/?geocode=%D1%83%D0%BB.%2B%D0%A9%D0%B5%D1%82%D0%B8%D0%BD%D0%BA%D0%B8%D0%BD%D0%B0%2C%2B33%2C%2B%D0%9D%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA


,short_name,full_name,address_for_url,naddress,latit,Longit
0,СГУВТ,Сибирский государственный университет водного ...,"ул.+Щетинкина,+33,+Новосибирск","ул. Щетинкина, 33, Новосибирск",55.028445,82.914546
1,СГУГиТ,Сибирский государственный университет геосисте...,"ул.+Плахотного,+10","ул. Плахотного, 10",54.986449,82.862857
2,НВИ,Новосибирский военный институт им. генерала ар...,"ул.+Ключ-Камышенское+Плато,+6/2","ул. Ключ-Камышенское Плато, 6/2",54.984026,83.035396
3,НГУАДИ,Новосибирский государственный университет архи...,"Красный+просп.,+38","Красный просп., 38",55.031541,82.9206
4,НГТУ,Новосибирский государственный технический унив...,"просп.+Карла+Маркса,+20","просп. Карла Маркса, 20",53.196488,50.145477


In [ ]:
univer.to_csv(path + "nov_univer.csv")

* **Муниципальные детские сады**

In [131]:
adr = list()
name = list()
main_kinder_url = "https://neagent.info/novosibirsk/kindergarten/municipalnye/"
html = urlopen(main_kinder_url)
soup = BeautifulSoup(html.read(), "html.parser")
table = soup.find("table", {"class": "district-markerlist"})
tr = table.findAll("tr")

for i in tr:
    div = i.find("div",{"class":"params-marker-desc"})
    if div:
        address = div.findAll("p")
        adr.append(address[0].text)
        name.append(address[1].text)
        
kinder = pd.DataFrame({"address":adr, "name":name})
kinder["address_for_url"] = "Новосибирск,+"+kinder["address"].str.replace(" ","+")
kinder = kinder.drop_duplicates()
kinder.head()

,address,name,address_for_url
0,"Дзержинский, ул. Фрунзе, 59/3",МКДОУ Детский сад №459 комбинированного вида,"Новосибирск,+Дзержинский,+ул.+Фрунзе,+59/3"
2,"Дзержинский, пр. Дзержинского, 59а",МКДОУ Детский сад №362 комбинированного вида,"Новосибирск,+Дзержинский,+пр.+Дзержинского,+59а"
3,"Дзержинский, ул. Толбухина, 31/1",МКДОУ Детский сад № 457 комбинированного вида,"Новосибирск,+Дзержинский,+ул.+Толбухина,+31/1"
5,"Дзержинский, ул. Есенина, 27а","МАДОУ Детский сад №373 комбинированного вида ""...","Новосибирск,+Дзержинский,+ул.+Есенина,+27а"
6,"Дзержинский, ул. Адриена Лежена, 21","МАДОУ Детский сад №373 комбинированного вида ""...","Новосибирск,+Дзержинский,+ул.+Адриена+Лежена,+21"


In [163]:
kinder = add_info_geo(kinder,"address_for_url")
kinder.head()

0
https://geocode-maps.yandex.ru/1.x/?geocode=%D0%9D%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA%2C%2B%D0%94%D0%B7%D0%B5%D1%80%D0%B6%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9%2C%2B%D1%83%D0%BB.%2B%D0%A4%D1%80%D1%83%D0%BD%D0%B7%D0%B5%2C%2B59/3


KeyError: 1

In [ ]:
kinder.to_csv(path + "nov_kinder.csv")